In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

class CustomFashionMNIST(Dataset):
    def __init__(self, root, train=True, transform=None):
        # Initialize and download FashionMNIST dataset
        self.dataset = datasets.FashionMNIST(root=root, train=train, download=True, transform=transform)

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.dataset)

    def __getitem__(self, idx):
        # Get the image and label for the sample at index `idx`
        image, label = self.dataset[idx]
        return image, label

# Define a transformation to convert images to tensors and normalize them
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Initialize dataset
train_data = CustomFashionMNIST(root='./data', train=True, transform=transform)
test_data = CustomFashionMNIST(root='./data', train=False, transform=transform)

# Create DataLoaders for batching
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)


In [20]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate model, define loss function and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [21]:
# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader)}")

# Save model weights
PATH = "fashion_mnist_model.pth"
torch.save(model.state_dict(), PATH)


Epoch 1, Loss: 0.43383923767885163
Epoch 2, Loss: 0.27664693676904323
Epoch 3, Loss: 0.22824137286940363
Epoch 4, Loss: 0.1934647657382272
Epoch 5, Loss: 0.16848122586668934
Model weights saved.


<ipython-input-22-f1ce5cbcca6b>:3: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Test Accuracy: 91.26%


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>